#### INST326 OOP Project 04

Rename this notebook, replacing "_Assignment" with "_YourName"<br>
Insert Signature Block Here

#### Christian Sorensen
> INST326
> Project 04
> 11/25/2024
#### Honor Pledge
> I pledge that the work contained in this assignment is my own, and that I have complied with University and course policies on academic integrity and AI use.


You may work as an individual on **ONE** of the following projects, **OR** if you want to work as a group, contact Dr. Dempwolf for a project assignment. That group assignment will be part of an ongoing research project analyzing innovation ecosystems. 

### Individual Projects
Choose **ONE** of the following projects and write to code solution in the code cell below your choice. Use comments in your code to document your solution. If you need to write comments to the grader, add a markdown cell immediately above your code solution and add your comments there. Be sure to read and follow the Notebook Instructions at the bottom of this notebook. Your grade may depend on it! 

#### 1. Library Management System
>  Objective: Develop a system to manage a library’s collection of books, users, and loan records. This system should allow users to borrow and return books, as well as track which books are currently available.
>
> Requirements
>>- Use classes to represent books, users, and the library.
>>- Implement encapsulation to protect class attributes.
>>- Use inheritance to handle different types of users (e.g., students and teachers).
>>- Demonstrate polymorphism in borrowing rules (e.g., different borrowing limits for students vs. teachers).
>>- Include methods for adding/removing books, registering users, and managing book loans.
>>- Include execution code to demonstrate that your solution works

In [ ]:
from __future__ import annotations
from abc import ABC


class Book:
    """Book class, with attributes for title, author, and year of publication"""
    def __init__(self, title: str, author: str = "Unknown", year: str = "Unknown"):
        self.title = title
        self.author = author
        self.year = year

    def __str__(self):
        return self.title

    def __repr__(self):
        return self.title
    
    def __eq__(self, book_2):
        """Compare if two books are the same"""
        if not isinstance(book_2, Book):
            raise TypeError("Cannot compare Book object with non-Book object")
        else:
            for attr in self.__dict__: # Checks if all the shared attributes between the books are identical, so that comparisons with LibraryBook objects also work
                if attr not in book_2.__dict__:
                    continue
                if getattr(self, attr) != getattr(book_2, attr):
                    return False # If the attributes do not match, return False
            return True
            
    
class LibraryBook(Book):
    """LibraryBook objects are used internally as 'templates', and hold information about a book in the library catalogue.
    Physical copies of books are tracked using unique sub-ids objects, since attributes of the LibraryBook itself are likely to be identical between them.
    
    This class makes it easy to assign internally managed attributes to the more user-friendly Book class"""
    def __init__(self,  book_id: int, title: str, author: str = "Unknown", year: str = "Unknown"):
        super().__init__(title, author, year)
        self.book_id = book_id # Unique identifier for book, for differentiating different versions of the same book, or books with the same title

    @property
    def book_id(self) -> str:
        return self.__book_id
    
    @book_id.setter
    def book_id(self, new_id: str):
        if type(new_id) is str and new_id.isdigit() and int(new_id) >= 0: # Ensure new id is a non-negative integer as a str
            self.__book_id = new_id
        else:
            raise ValueError("Id must be non-negative integer in str type")


class User(ABC):
    """Base User abstract class. Subclasses can implement restrictions on loaning books"""
    def __init__(self, user_id: int, firstname: str, lastname: str):
        self.user_id = user_id # Unique identification number to differentiate between users with identical names
        self.fname = firstname
        self.lname = lastname
        self.loaned_books = {} # Holds the full ids of every book current loaned
        self.loan_limit = 0

    def __repr__(self):
        """Object is represented by lastname, firstname"""
        return f"{self.lname}, {self.fname}"
    
    @property
    def name(self):
        """Read only. Just a convenient way to get users name in <fname lname> format"""
        return f"{self.fname} {self.lname}"

    @property
    def user_id(self) -> str:
        return self.__user_id
    
    @user_id.setter
    def user_id(self, new_id: str):
        """Ensure new id is a non-negative integer in string format"""
        if type(new_id) is str and new_id.isdigit() and int(new_id) >= 0:
            self.__user_id = new_id
        else:
            raise ValueError("Id must be non-negative integer in str type")
        
    def print_loaned_books(self):
        """Prints a list of the books the user is currently reserving"""
        for _, (full_id, book) in enumerate(self.loaned_books.items()):
            print(f"{full_id}: {book.title}")


class Student(User):
    """Student class. Has a loan limit of 10"""
    def __init__(self, user_id: int, firstname: str, lastname: str):
        super().__init__(user_id, firstname, lastname)
        self.loan_limit = 10


class Teacher(User):
    """Teacher class. Has a loan limit of 20"""
    def __init__(self, user_id: int, firstname: str, lastname: str):
        super().__init__(user_id, firstname, lastname)
        self.loan_limit = 10



class LibraryManager:
    def __init__(self):
        self.__users = {} # Each user is stored as a key value pair, with the key being their user_id and the value being the user object
        self.__catalogue = {} # Stores each library book by its book_id
        self.__loans = {} # Each key is a book_id associated with a subdictictionary, with each key in that dictionary being the sub_id of a copy of that book, and the value of that key being the user_id of the user currently reserving that book
        self.__book_id_len = 8 # For formatting the different IDs. book_ids are formatted as 'NNNNNNNN'
        self.__sub_id_len = 3 # sub_ids are formatted as 'NNN', making full_ids 'NNNNNNNN-NNN'
        self.__user_id_len = 8 # user_ids are formatted as 'NNNNNNNN'

    ######################   PRIVATE METHODS   ################################################################

    def __add_copy(self, book: LibraryBook) -> str:
        """Adds a copy of the book to the loan pool. Returns the sub_id of the new copy"""
        book_id = book.book_id
        sub_id = 0
        while str(sub_id).zfill(self.__sub_id_len) in self.__loans[book_id]: # Gets the lowest available sub_id
            sub_id += 1
        sub_id = str(sub_id).zfill(self.__sub_id_len)
        self.__loans[book_id].update({sub_id:None})
        book = self.get_book_from_id(book_id)
        return sub_id

    def __get_user_info(self, user_or_id: User | str) -> tuple[str, LibraryBook]:
        """Returns a tuple of (user_id, user). More efficient, but less intuitive so the other methods are there too. Makes it simpler to allow methods to accept either user_id or a user object"""
        if isinstance(user_or_id, User):
            user = user_or_id
            user_id = user.user_id
            if user_id not in self.__users:
                raise ValueError(f"User has not been added to the library ({user_or_id})")
        elif isinstance(user_or_id, str):
            user_id = user_or_id
            if user_id not in self.__users:
                raise ValueError(f"User does not exist ({user_or_id})")
            user = self.__users[user_id]
        else:
            raise TypeError("user_or_id must be a User child object or a valid user_id")
        return (user_id, user)
    
    def __get_book_info(self, book_or_id: Book | str) -> tuple[str, User]:
        """Returns a tuple of (book_id, book). Makes it simpler to allow methods to accept either book_id or a book object"""
        if isinstance(book_or_id, Book):
            book = book_or_id
            book_id = self.get_id_from_book(book)
            if book_id is None:
                raise ValueError("Book not in library catalogue, please add the book first")
        elif isinstance(book_or_id, str):
            book_id = book_or_id
            if '-' in book_id: # Convert full ids to just the book id
                book_id = book_id.split('-')[0]
            if book_id not in self.__catalogue:
                raise ValueError(f"Book does not exist in catalogue ({book_id})")
            book = self.__catalogue[book_id]
        else:
            raise TypeError(f"book_or_id must be a Book or LibraryBook object, or a valid book_id ({book_or_id})")
        return (book_id, book)
    
    def __is_book_available(self, book_or_id: Book | str) -> str | bool:
        """Checks if a book has an available copy. Returns the sub_id of the available copy, or False if unavailable.
        If a full_id is provided, only checks the availability of that copy"""
        if isinstance(book_or_id, str) and '-' in book_or_id: # If a full_id was given, only check availability of the specific copy
            full_id = book_or_id
            book_id, sub_id = full_id.split('-')
            if book_id not in self.__loans: # Check for errors
                raise ValueError(f"Book does not exist in loan pool ({book_id})")
            elif sub_id not in self.__loans[book_id]:
                raise ValueError(f"Copy does not exist in loan pool ({full_id})")
            if self.__loans[book_id][sub_id]: # Check if the copy is currently loaned out
                return False
            else:
                return True
        book_id, _ = self.__get_book_info(book_or_id)
        loan_dict = self.__loans[book_id]
        for _, (sub_id, loaned_to) in enumerate(loan_dict.items()): # Checks each sub_id to see if an available copy exists
            if loaned_to == None:
                return sub_id
        return False
    

    ######################   PUBLIC METHODS   ################################################################

    ####### REGISTER USERS AND BOOKS ###############################

    def add_student(self, first_name: str, last_name: str):
        """Adds a new student to the system"""
        user_id = 0
        while str(user_id).zfill(self.__user_id_len) in self.__users:
            user_id += 1
        user_id = str(user_id).zfill(self.__user_id_len)
        student = Student(user_id, first_name, last_name)
        self.__users.update({user_id:student}) # Adds student to the users dict, with their user_id as the key
        print(f"Added {student} as 'Student' ({user_id})")
        return student

    def add_teacher(self, first_name: str, last_name: str):
        """Adds a new teacher to the system"""
        user_id = 0
        while str(user_id).zfill(self.__user_id_len) in self.__users: # Gets the lowest available id
            user_id += 1
        user_id = str(user_id).zfill(self.__user_id_len)
        teacher = Teacher(user_id, first_name, last_name)
        self.__users.update({user_id:teacher}) # Adds teacher to the users dict, with their user_id as the key
        print(f"Added {teacher} as 'Teacher' ({user_id})")
        return teacher

    def add_library_book(self, book: Book):
        """Adds a new book to the catalogue and make a copy available for loans. If the book already exists in the catalogue, it just makes a new copy"""
        for index, (book_id, library_book) in enumerate(self.__catalogue.items()): # Checks if the book has already been registered in the catalogue before
            if book == library_book:
                sub_id = self.__add_copy(library_book) # Adds a new copy of the book
                print(f"Added new copy of '{book.title}' to loan pool ({book_id}-{sub_id})")
                return
        book_id = 0 # If the book is not already in the catalogue, generates a new id for it
        while str(book_id).zfill(self.__book_id_len) in self.__catalogue: # Gets the lowest available book_id
            book_id += 1
            if book_id > 10*self.__book_id_len:
                raise OverflowError("No book_ids left. Please remove books before registering more")
        book_id = str(book_id).zfill(self.__book_id_len)
        library_book = LibraryBook(book_id, book.title, book.author, book.year)
        book_id = library_book.book_id
        self.__catalogue.update({book_id:library_book}) # Adds the book to the catalogue
        self.__loans.update({book_id:{}})
        self.__add_copy(library_book)
        print(f"Added '{library_book.title}' to catalogue ({book_id})")

    def remove_user(self, user_or_id: User | str, force: bool = False):
        """Remove a user from the library system. Raises error if the user is still holding on to any books, unless force is set to True."""
        user_id, user = self.__get_user_info(user_or_id)
        if len(user.loaned_books) > 0 and force == False:
            raise Exception(f"{user} still has reserved books. Please return books before removing, or set 'force' to true to override")
        self.__users.pop(user_id) # Remove user from users dict
        for full_id in user.loaned_books: # Unassign the user from each book they're currently reserving
            self.return_book(user_id, full_id)
        print(f"{user} has been removed from the library system")

    def remove_book(self, book_or_id: Book | str, force: bool | str = False) -> bool:
        """Removes a book or copy from the library catalogue. Raises error if a copy is currently loaned out, unless force is set to True.
        If force is set to 'available', only removes copies that are not currently being loaned. Returns True if the book was removed, or False if skipped"""
        if force not in {True, False, 'available'}: # Ensure force is properly set
            raise ValueError(f"'force' must be set to True, False, or 'available' ({force})")
        if isinstance(book_or_id, str) and '-' in book_or_id: # Check if a full_id was given
            full_id = book_or_id
            book_id, sub_id = full_id.split('-')
            book_id, book = self.__get_book_info(book_id)
            if not self.__is_book_available(full_id) and force != True:
                if force == False: # If copy is loaned out and force is not set to True or 'available'
                    raise Exception(f"{full_id} ('{book.title}') is currently reserved. Please return before removing, or set 'force' to true to override")
                elif force == 'available':
                    print(f"Skipping {full_id} as it is currently reserved")
                    return False
            else:
                self.__loans[book_id].pop(sub_id) # Remove copy from dict
                print(f"Removed {full_id}")
                if len(self.__loans[book_id]) < 1:
                    self.__catalogue.pop(book_id)
                    print(f"No copies left. Removed {book_id} ('{book.title}') from library catalogue")
                return True
        else:
            book_id, book = self.__get_book_info(book_or_id)
            print(f"Removing {book_id} ('{book.title}') from library catalogue")
            remove_list = [] # List of full_ids to be removed
            copies_removed = 0
            num_copies = len(self.__loans[book_id])
            for copy in self.__loans[book_id]: # Check if any of the copies are being loaned out
                full_id = f"{book_id}-{copy}"
                if not self.__is_book_available(full_id) and force == False: 
                    raise Exception(f"{book_id} ('{book.title}') still has reserved copies. Please return them before removing. Set force to True to override, or to 'available' to only remove available copies")
                else:
                    remove_list.append(full_id) # Set the book to be removed (or skipped if not available and force is not True)
            for full_id in remove_list:
                was_removed = self.remove_book(full_id, force) # Remove the book, skipping if force is set to 'available'
                if was_removed:
                    copies_removed += 1
            if copies_removed < 1:
                print(f"No available copies to remove")
                return
            elif copies_removed < num_copies:
                print(f"Removed {copies_removed} out of {num_copies} copies of {book_id} ('{book.title}') from catalogue. {num_copies - copies_removed} copies are currently reserved and have been skipped.")
                

    ################################################################

    ####### LOAN BOOKS, RETURN BOOKS, AND VIEW CATALOGUE ###########
    
    def print_catalogue(self):
        """Prints a catalogue showing every registered book in the library system, as well as the number of copies"""
        print_list = []
        for _, (book_id, book) in enumerate(self.__catalogue.items()):
            title = book.title # The books title
            num_copies = len(self.__loans[book_id]) # The total number of copies
            num_available = 0
            for _, (sub_id, loaned_to) in enumerate(self.__loans[book_id].items()):
                if loaned_to is None: # If the copy is currently assigned to None
                    num_available += 1
            print_list.append((title, num_copies, num_available))
        for book in print_list:
            print(f"{book[0]}: {book[2]} available copies out of {book[1]}")

    def show_availability(self, book_or_id: Book | str) -> bool:
        """Print the availability of a book if a book object or book_id is given or the availability of a specific copy if a full_id is given"""
        if isinstance(book_or_id, str) and '-' in book_or_id: # If a full_id is given
            full_id = book_or_id
            book_id, sub_id = full_id.split('-')
            if sub_id not in self.__loans[book_id]:
                print(f"Invalid sub_id ({sub_id})")
                return
            if self.__loans[book_id][sub_id]:
                print(f"{full_id} is not currently available")
                return False
            else:
                print(f"{full_id} is currently available")
                return True
        else: # If only a book or book_id is given
            book_id, book = self.__get_book_info(book_or_id)
            available_copies = []
            loan_dict = self.__loans[book_id]
            for _, (sub_id, loaned_to) in enumerate(loan_dict.items()): # Checks each sub_id to see if an available copy exists
                if loaned_to == None:
                    available_copies.append(sub_id)
            if len(available_copies) > 0: # If at least one available copy was found
                print(f"{len(available_copies)} available copies for '{book.title}':")
                tab = "    "
                for c in available_copies: # Print indented list of each full_id
                    print(f"{tab}{book_id}-{c}")
                return True # Return true if an available copy exist
            else:
                print(f"No available copies for '{book.title}'")
                return False

    def loan_book(self, user_or_id: User | str, book_or_id: Book | str) -> str | None:
        """Reserves a book to a given user. Returns the full_id of the loaned book. Returns full_id of loaned book"""
        book_id, book = self.__get_book_info(book_or_id)
        user_id, user = self.__get_user_info(user_or_id)
        if len(user.loaned_books) >= user.loan_limit: # If the user has met their loan limit
            print(f"User '{user}' is loaning too many books. Please return a book before reserving more.")
            return
        available_copy = self.__is_book_available(book)
        if available_copy:
            full_id = f"{book_id}-{available_copy}" # Gets the full ID of the book in the format of <book_id>-<sub_id>
            user.loaned_books.update({full_id:book})
            self.__loans[book_id].update({available_copy:user_id}) # Assigns the user's user_id to the sub_id of the copy they're reserving
            print(f"Loaned {full_id} ('{book.title}') to {user}")
            return full_id

    def return_book(self, user_or_id: User | str, book_or_id: Book | str):
        """Returns a copy of a book to the available loan pool"""
        user_id, user = self.__get_user_info(user_or_id)
        if isinstance(book_or_id, str) and '-' in book_or_id: # Check if a full_id was provided, to return a specific copy
            full_id = book_or_id.split('-')
            book_id = full_id[0]
            sub_id = [1]
            book = self.get_book_from_id(book_id)
            if book_or_id not in user.loaned_books:
                print(f"'{book}' not in user {user}'s reserved books")
                return
        else: # If a full id was not given, check if the user is currently loaning a copy of the book, and select the first copy for return
            book_id, book = self.__get_book_info(book_or_id)
            for _, (current_full_id, book) in enumerate(user.loaned_books.items()):
                current_book_id, current_sub_id = current_full_id.split('-')
                if current_book_id == book_id:
                    sub_id = current_sub_id
                    user.loaned_books.pop(f"{book_id}-{sub_id}") # Remove the book from the user's loan list
                    self.__loans[book_id].update({sub_id:None}) # Unassigns that user from the copy
                    print(f"{book_id}-{sub_id} ({book}) has been returned")
                    return
            print(f"'{book}' not in user {user}'s reserved books") # If all reserved books were checked and no matches were found

    ################################################################

    ####### CONVERT TO AND FROM ID IF NECESSARY ####################

    def get_user_from_id(self, user_id: str) -> User:
        """Gets a user object from their user_id"""
        if user_id in self.__users:
            return self.__users[user_id]
        else:
            raise ValueError(f"User does not exist ({user_id})")
        
    def get_id_from_book(self, book: Book) -> str | None:
        """Gets the LibraryBook id for a given book, or None if the book is not in the catalogue"""
        for _, (book_id, library_book) in enumerate(self.__catalogue.items()): # Checks each book in the catalogue
            if book == library_book: # Checks if the Book object matches the current LibraryBook object
                return book_id
        return None
    
    def get_book_from_id(self, book_id: str) -> LibraryBook:
        """Returns the LibraryBook object from a given id"""
        if '-' in book_id:
            book_id = book_id.split('-')[0] # Converts full_ids to just the book_id
        if book_id in self.__catalogue:
            return self.__catalogue[book_id]
        else:
            print(f"book_id does not exist in catalogue ({book_id})")
    
        

# Test code
if __name__ == "__main__":

    ### HOW TO USE ###
    # 1. create book objects
    # 2. add the book objects to the LibraryManager with add_library_book()
    # 3. add students or teachers to the LibraryManager with add_student() or add_teacher()
    # 4. loan books with LibraryManager.loan_book()
    # 5. return books with LibraryManager.return_book()

    library = LibraryManager()
    books = [] # Add some books to a list
    books.append(Book("To Kill A Mockingbird", "Harper Lee", "1960"))
    books.append(Book("The Great Gatsby"))
    books.append(Book("Kite Runner"))
    books.append(Book("Harry Potter"))

    print("Registering books")
    from random import randint
    for book in books:
        for i in range(randint(1,5)): # Add a random number of copies between 1 and 5
            library.add_library_book(book)
    
    print("")
    print("Registering users")
    student = library.add_student("Robert", "Parr") # Add a student
    teacher = library.add_teacher("Kevin", "Davern") # Add a teacher

    print("") # Just an empty line to make reading the test code easier
    print("Printing catalogue")

    library.print_catalogue() # Show the books in the catalogue

    print("")
    print("Getting book availability")

    library.show_availability(books[0]) # Show the availability of each book
    library.show_availability(books[3])

    print("")
    print("Loaning books")

    book_1 = library.loan_book(student, books[0]) # Loan some books
    book_2 = library.loan_book(teacher, books[3])

    print("")
    print("Showing that the books have been reserved")
    
    library.show_availability(books[0]) # Show that the number of available copies has decreased
    library.show_availability(book_2) # Check the availability of the specific copy that was loaned out

    print("")
    print(f"{student}'s books: ")
    student.print_loaned_books() # Show the book ids in each users 

    print(f"{teacher}'s books: ")
    teacher.print_loaned_books()

    print("")
    print("Returning book")

    library.return_book(student, books[0]) # Return a book

    print("")
    print("Showing that the book has been returned")

    library.show_availability(books[0])

    library.print_catalogue() # Show that the copy has been made available again

Registering books
Added 'To Kill A Mockingbird' to catalogue (00000000)
Added 'The Great Gatsby' to catalogue (00000001)
Added new copy of 'The Great Gatsby' to loan pool (00000001-001)
Added new copy of 'The Great Gatsby' to loan pool (00000001-002)
Added 'Kite Runner' to catalogue (00000002)
Added 'Harry Potter' to catalogue (00000003)
Added new copy of 'Harry Potter' to loan pool (00000003-001)
Added new copy of 'Harry Potter' to loan pool (00000003-002)
Added new copy of 'Harry Potter' to loan pool (00000003-003)

Registering users
Added Parr, Robert as 'Student' (00000000)
Added Davern, Kevin as 'Teacher' (00000001)

Printing catalogue
To Kill A Mockingbird: 1 available copies out of 1
The Great Gatsby: 3 available copies out of 3
Kite Runner: 1 available copies out of 1
Harry Potter: 4 available copies out of 4

Getting book availability
1 available copies for 'To Kill A Mockingbird':
    00000000-000
4 available copies for 'Harry Potter':
    00000003-000
    00000003-001
    00

#### 2. Online Shopping Cart System
>  Objective: Build a shopping cart system for an online store that manages products, shopping carts, and orders.
>
> Requirements
>>- Use classes to represent products, shopping carts, and orders.
>>- Implement encapsulation to handle product prices and cart contents securely.
>>- Use inheritance to create different types of products (e.g., electronics, clothing).
>>- Demonstrate polymorphism by calculating discounts based on product type.
>>- Include execution code to demonstrate that your solution works

In [ ]:
# Solution - enter your code solution below


#### 3. Restaurant Reservation System
>  Objective: Create a reservation system for a restaurant that manages tables, reservations, and customers.
>
>  Requirements
>>- Use classes to represent tables, customers, and reservations.
>>- Implement encapsulation for managing table availability and reservation details.
>>- Use inheritance to differentiate between walk-in and advance reservations.
>>- Demonstrate polymorphism by handling special cases (e.g., priority seating for VIP customers).
>>- Include execution code to demonstrate that your solution works

In [ ]:
# Solution - enter your code solution below


#### 4. Vehicle Rental System
>  Objective: Develop a vehicle rental system that manages a fleet of vehicles, customer rentals, and payment processing.
>
>  Requirements
>>- Use classes to represent different types of vehicles, customers, and rental transactions.
>>- Implement encapsulation to handle sensitive information like customer payment details.
>>- Use inheritance to differentiate between various vehicle types (e.g., cars, trucks, motorcycles).
>>- Demonstrate polymorphism by applying different rental pricing strategies based on vehicle type.
>>- Include execution code to demonstrate that your solution works

In [ ]:
# Solution - enter your code solution below


#### 5. Online Learning Platform
>  Objective: Create an online learning platform that manages courses, students, and instructors.
>  
>  Requirements
>>- Use classes to represent courses, students, and instructors.
>>- Implement encapsulation to manage sensitive information like student grades.
>>- Use inheritance to handle different types of courses (e.g., free, paid, and premium).
>>- Demonstrate polymorphism in applying different grading schemes for assignments.
>>- Include execution code to demonstrate that your solution works

In [ ]:
# Solution - enter your code solution below


#### 6. E-Commerce Order Processing System
>  Objective: Build an order processing system for an online store that manages products, customers, and orders.
>  
>  Requirements
>>- Use classes to represent products, customers, and orders.
>>- Implement encapsulation for handling payment details securely.
>>- Use inheritance for different types of products (e.g., physical goods, digital downloads).
>>- Demonstrate polymorphism by applying different shipping costs based on product type.
>>- Include execution code to demonstrate that your solution works

In [ ]:
# Solution - enter your code solution below


### Notebook Instructions
> Before turning in your notebook:
> 1. Make sure you have renamed the notebook file as instructed
> 2. Make sure you have included your signature block and that it is correct according to the instructions
> 3. comment your code as necessary
> 4. run all code cells and double check that they run correctly. Include you execution code in your submission. If you can't get your code to run correctly and you want partial credit, add a note for the grader in a new markdown cell directly above your code solution.<br><br>
Turn in your notebook by uploading it to ELMS<br>
IF the exercises involve saved data files, put your notebook and the data file(s) in a zip folder and upload the zip folder to ELMS